# Hearthstone Deck Recommendation
### Objetivo: Construir un mazo ganador para una partida de Hearthstone.

## Librerias

In [341]:
import pandas as pd
import numpy as np
import json
import random as rd
rd.seed(3633)
from collections import Counter
import keras
import pickle


## Base de datos
La base de datos se obtuvo de la plataforma HearthPwn (https://www.hearthpwn.com/decks), esta contiene información sobre mazos creados por jugadores de Hearthstone. Los datos incluyen la clase del mazo, sus 30 cartas, el coste de creación, la puntuación, entre otros, pero dentro de las columnas más importantes está el rating que asignaron múltiples jugadores en la plataforma.

In [314]:
df = pd.read_csv("data/data.csv")
df.head()

,craft_cost,date,deck_archetype,deck_class,deck_format,deck_id,deck_set,deck_type,rating,title,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
0,9740,2016-02-19,Unknown,Priest,W,433004,Explorers,Tavern Brawl,1,Reno Priest,...,374,2280,2511,2555,2566,2582,2683,2736,2568,2883
1,9840,2016-02-19,Unknown,Warrior,W,433003,Explorers,Ranked Deck,1,RoosterWarrior,...,1781,1781,2021,2021,2064,2064,2078,2510,2729,2736
2,2600,2016-02-19,Unknown,Mage,W,433002,Explorers,Theorycraft,1,Annoying,...,1793,1801,1801,2037,2037,2064,2064,2078,38710,38710
3,15600,2016-02-19,Unknown,Warrior,W,433001,Explorers,NaN,0,Standart pay to win warrior,...,1657,1721,2018,2296,2262,336,2729,2729,2736,2760
4,7700,2016-02-19,Unknown,Paladin,W,432997,Explorers,Ranked Deck,1,Palamix,...,2027,2029,2029,2064,2078,374,2717,2717,2889,2889


In [315]:
df.describe()

,craft_cost,deck_id,rating,card_0,card_1,card_2,card_3,card_4,card_5,card_6,...,card_20,card_21,card_22,card_23,card_24,card_25,card_26,card_27,card_28,card_29
count,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,...,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000,346232.000000
mean,5645.360218,394152.425798,2.687230,166.555443,215.682814,328.068948,388.254399,465.439497,530.422084,618.649576,...,6676.817657,7879.285156,9310.508977,10780.480210,12401.890674,13709.898080,15068.109406,15955.823881,19239.277147,20537.999847
std,3927.986295,222605.617140,22.117751,515.546751,549.163776,609.109069,629.218897,710.534945,813.475129,987.250288,...,12610.593768,13750.165455,14885.066464,15862.422929,16731.209629,17295.441879,17766.351821,18039.395857,18567.676290,18686.462303
min,0.000000,18.000000,0.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
25%,2720.000000,216721.500000,1.000000,64.000000,75.000000,189.000000,237.000000,279.000000,304.000000,401.000000,...,1158.000000,1363.000000,1659.000000,1783.000000,1794.000000,1913.000000,2010.000000,2037.000000,2078.000000,2095.000000
50%,5000.000000,406046.500000,1.000000,138.000000,180.000000,285.000000,315.000000,415.000000,475.000000,559.000000,...,1940.000000,2029.000000,2061.000000,2078.000000,2275.000000,2488.000000,2577.000000,2682.000000,2901.000000,3015.000000
75%,7740.000000,590820.500000,1.000000,238.000000,285.000000,421.000000,476.000000,605.000000,643.000000,763.000000,...,2610.000000,2736.000000,2890.000000,2958.000000,38391.000000,38526.000000,38727.000000,38833.000000,38918.000000,39034.000000
max,48000.000000,749548.000000,4016.000000,41409.000000,41409.000000,41609.000000,41409.000000,41409.000000,41609.000000,41609.000000,...,41609.000000,41609.000000,41609.000000,41609.000000,41609.000000,41609.000000,41841.000000,42146.000000,42146.000000,42146.000000


In [316]:
df['deck_class'].value_counts()

deck_class
Mage       45306
Priest     44307
Paladin    42266
Warlock    38022
Druid      37891
Shaman     36457
Warrior    35944
Rogue      34794
Hunter     31245
Name: count, dtype: int64

La clase de mazo con más entradas es "Mage". Para la implementación del sistema de recomendación utilizaremos solo mazos mago.

In [317]:
df = df[df['deck_class'] == 'Mage']

## Cartas y mazos
Definimos el conjunto *cards* con las *N* cartas más repetidas en los mazos mago y *D* como el tamaño del mazo para armar.

In [318]:
cards_cols = df.iloc[:, 11:41] # columnas con las cartas en la base de datos
all_cards = (cards_cols.values.ravel()).tolist() # lista con las cartas
counter = Counter(all_cards) # contador de las cartas

N = 40 # tamaño conjunto cartas para elegir
cards =  [item[0] for item in counter.most_common(N)] # conjunto con las N cartas más repetidas

D = 30 # tamaño mazo Hearthstone

El mazo lo definimos como un vector binario *deck* con *N* entradas y exactamente *D* iguales a 1. Este vector nos indica con la i-esima entrada si está o no (1 o 0) en el mazo la i-esima carta de *cards*. Definimos la función *binary_to_cards* que transforma el vector *deck* en una lista con la enumeración de las cartas del mazo. A continación se presenta un ejemplo de mazos.

In [319]:
def binary_to_cards(deck, cards):
    return [int(x) for x in deck*cards if x != 0]

In [320]:
deck = np.zeros(N, dtype=int) # definir mazo jugador
deck[np.random.choice(N, D, replace=False)] = 1 # generar mazo aleatorio jugador
print(f'Vector mazo: {deck}')
print(f'Cartas mazo: {binary_to_cards(deck, cards)}')

Vector mazo: [1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 0 1]
Cartas mazo: [662, 315, 1004, 405, 614, 77, 825, 564, 1783, 457, 192, 38710, 172, 2874, 621, 138, 1080, 1927, 1793, 2572, 2262, 251, 113, 749, 2875, 1928, 430, 2037, 1087, 2078]


## Transiciones y acciones
El proceso para mejorar el mazo *deck* consta de aplicar múltiples veces la función de transición *transition* determinada por una acción. Una acción *action* es una tupla con las cartas del reemplazo que modifica exactamente una carta del mazo por una carta que actualmente no está incluida. A continuación, un ejemplo del uso de la función *transition*, en donde se elige de forma aleatoria la acción.

In [321]:
def transition(deck, action):
    deck[action[0]] = 0 # eliminamos la carta antigua del mazo
    deck[action[1]] = 1 # agregamos la carta nueva al mazo
    return deck

In [322]:
print(f'Mazo antiguo: {deck}')
print(f'Cartas mazo antiguo: {binary_to_cards(deck, cards)}')

old_card = np.random.choice(np.where(deck == 1)[0]) # elegimos aleatoriamente una carta que está en el mazo
new_card = np.random.choice(np.where(deck == 0)[0]) # elegimos aleatoriamente una carta que no está en el mazo
action = (old_card, new_card) # definimos la acción

print()
print(f'Acción: ({action[0]},{action[1]})')
print()

new_deck = transition(deck, action)
print(f'Mazo nuevo: {new_deck}')
print(f'Cartas mazo nuevo: {binary_to_cards(new_deck, cards)}')

Mazo antiguo: [1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 0 1]
Cartas mazo antiguo: [662, 315, 1004, 405, 614, 77, 825, 564, 1783, 457, 192, 38710, 172, 2874, 621, 138, 1080, 1927, 1793, 2572, 2262, 251, 113, 749, 2875, 1928, 430, 2037, 1087, 2078]

Acción: (35,2)

Mazo nuevo: [1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 0 1
 1 0 1]
Cartas mazo nuevo: [662, 315, 555, 1004, 405, 614, 77, 825, 564, 1783, 457, 192, 38710, 172, 2874, 621, 138, 1080, 1927, 1793, 2572, 2262, 251, 113, 749, 2875, 1928, 2037, 1087, 2078]


## Win Rate
Para elegir las acciones que nos lleven a encontrar un mazo ganador usaremos el *win_rate* de este mazo. Entonces, para elegir la próxima acción buscaremos cual es la que maximiza el *win_rate* del mazo.

Para hacer una función predictora del *win_rate* entrenamos distintos modelos de machine learning, esto se realizó en el notebook *win_rate_prediction.ipynb*. A continuación, se importan estos modelos ya entrenados.

In [323]:
model = keras.models.load_model("win_rate_NN.keras") # importación modelo red neuronal
#model.summary()

with open("encoder", "rb") as f:
    encoder = pickle.load(f)
with open("win_rate_RF.pkl", "rb") as f:
    rf = pickle.load(f) # importación modelo random forest

Ahora, definimos la función *win_rate* que recibe el mazo en el formato enumerado de cartas y dependiendo del modelo elegido entrega el valor del win rate.

In [325]:
def win_rate(deck, encoder = encoder, model = model):
  mazo_gen_num = pd.DataFrame([{f"card_{i}": (deck[i] if i>-1 else 5645) for i in range(-1,30)}]).rename(columns={"card_-1":"craft_cost"})
  hero = "Mage"
  mazo_gen_cat = pd.DataFrame([{"hero":hero}])
  encoded_categorical = encoder.transform(mazo_gen_cat)
  # # Step 3: Convert the encoded data back to a DataFrame
  encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out())
  # # Step 4: Concatenate the numerical column with the one-hot encoded categorical columns
  final_df = pd.concat([mazo_gen_num, encoded_df], axis=1).rename(columns={"craft_cost":"dust"})
  if isinstance(model,keras.src.models.sequential.Sequential):
    results = model.predict(final_df,verbose=None)
  else:
    results = model.predict(final_df)
  if isinstance(results[0],np.ndarray):
    return results[0][0]
  else:
    return results[0]

Finalmente, imprimimos el win rate aproximado para el mazo *deck_p* usando los dos modelos. Primero en base a la red neuronal entrenada y después mediante un random forest.

In [326]:
print('Neural network:', win_rate(binary_to_cards(deck, cards))) # red neuronal
print('Random forest:', win_rate(binary_to_cards(deck, cards),model=rf)) # random forest

Neural network: 54.95451
Random forest: 49.72733333333331


## Implementación 

Con la función de *win_rate* ya definida podemos decidir cual es la siguiente acción más conveniente a realizar. Vamos a comenzar con un mazo *deck* y realizar acciones hasta que ya no sea conveniente seguir modificando el mazo, estas serán a los más *D*, ya que a lo más cambiamos todo el mazo para encontrar el óptimo. A continuación, definimos la función *next_action* que a partir del mazo actual encuentra la siguiente acción más conveniente a realizar según el *win_rate*, incluyendo la acción de preservar el mazo igual.

In [327]:
def next_action(deck, cards):
    current_win_rate = win_rate(binary_to_cards(deck, cards))
    best_win_rate = current_win_rate
    best_action = None
    
    zeros = [index for index, value in enumerate(deck) if value == 0]
    ones = [index for index, value in enumerate(deck) if value == 1]
     
    for zero in zeros:
        for one in ones: 
            temp_deck = deck.copy()  
            action = (one, zero)
            temp_deck = transition(temp_deck, action)
            temp_win_rate = win_rate(binary_to_cards(temp_deck, cards))
            if temp_win_rate > best_win_rate:
                best_win_rate = temp_win_rate
                best_action = action
    
    return best_action    

Ahora con la función *deck_reck* podemos hacer efectiva la mejor acción sucesivamente hasta llegar a *D* acciones o haber generado un mazo óptimo.

In [328]:
def deck_rec(deck, cards):
    for i in range(D):
        action = next_action(deck, cards)
        if action == None:
            break
        else:
            deck = transition(deck, action)
    return deck

In [329]:
print(f'Mazo inicial: {binary_to_cards(deck, cards)}')
print(f'Win rate inicial: {win_rate(binary_to_cards(deck, cards))}')
new_deck = deck_rec(deck, cards)
print(f'Mazo final: {binary_to_cards(new_deck, cards)}')
print(f'Win rate final: {win_rate(binary_to_cards(new_deck, cards))}')

Mazo inicial: [662, 315, 555, 1004, 405, 614, 77, 825, 564, 1783, 457, 192, 38710, 172, 2874, 621, 138, 1080, 1927, 1793, 2572, 2262, 251, 113, 749, 2875, 1928, 2037, 1087, 2078]
Win rate inicial: 54.95450973510742
Mazo final: [315, 555, 1004, 405, 614, 77, 825, 564, 1783, 457, 192, 38710, 172, 2874, 621, 138, 1080, 1659, 1927, 1793, 2572, 2262, 251, 113, 749, 2875, 1928, 2037, 1087, 2078]
Win rate final: 60.91288375854492


## Comparación de resultados
### Distintos Métodos
Vamos a comparar nuestra implementación con los siguientes métodos.

1. Random:
Entrega de forma aleatoria un mazo de cartas de la base de datos.

In [330]:
def random_deck():
  row = df.sample(n=1)
  id = row['deck_id'].tolist()[0]
  cards_cols = row.iloc[:, 11:41]
  cards_list = (cards_cols.values.ravel()).tolist() 
  return id, cards_list

2. Most popular: 
Retorna el mazo con el mejor rating de la base de datos.

In [331]:
def most_popular_deck():
  row = df[df['rating'] == df['rating'].max()]
  cards_cols = row.iloc[:, 11:41]
  cards_list = (cards_cols.values.ravel()).tolist() 
  return cards_list

3. Best Similar:
Recibe un mazo y retorna un mazo similar de la base de datos con mejor rating. 

Para definir la similitud, se implementó una función *difference* que calcula la cantidad de cartas diferentes entre los mazos. De esta forma, el modelo también recibe como parámetro un *delta* que define la máxima posible diferencia entre el mazo retornado y el original.

In [332]:
def difference(deck_1, deck_2):
  #deck_1 = df[df['deck_id'] == id_1].iloc[:, 11:41].values.flatten()
  #deck_2 = df[df['deck_id'] == id_2].iloc[:, 11:41].values.flatten()
  common = len(np.intersect1d(deck_1, deck_2))
  diff = len(deck_1) - common
  return diff

def best_similar_deck(my_deck, delta):
  best_decks = df[df['rating'] > 2]
  best_deck_id = -1
  best_deck_rating = -1
  for index, row in best_decks.iterrows():
    new_deck_id = row['deck_id']
    new_deck_rating = row['rating']
    deck_2 = df[df['deck_id'] == new_deck_id].iloc[:, 11:41].values.flatten()
    if difference(my_deck, deck_2) <= delta:
      #best_deck_rating = df.loc[df['deck_id'] == best_deck_id, 'rating'].values[0]
      if new_deck_rating >= best_deck_rating:
        best_deck_id = new_deck_id
        best_deck_rating = new_deck_rating
  if best_deck_id>=0:
    row = df[df['deck_id'] == best_deck_id]
    cards_cols = row.iloc[:, 11:41]
    cards_list = (cards_cols.values.ravel()).tolist() 
    return cards_list
  else:
    return my_deck

Entonces, a continuación vamos a seleccionar de forma aleatoria un mazo de la base de datos y vamos a comparar los distintos sistemas recomendadores usando el *win_rate*.

In [333]:
original_id, original_deck = random_deck() # generamos mazo aleatorio
print(f'Mazo inicial: {original_deck}')
print(f'Win rate inicial: {win_rate(original_deck)}')

Mazo inicial: [113, 195, 315, 315, 405, 405, 555, 555, 564, 614, 614, 662, 662, 749, 825, 1080, 1084, 1783, 1783, 1927, 2064, 2064, 2078, 2275, 2275, 2572, 2572, 2875, 38710, 38710]
Win rate inicial: 54.78474044799805


In [334]:
id_random, deck_random = random_deck() # generamos recomendación aleatoria
print(f'Recomendación random: {deck_random}')
print(f'Win rate random: {win_rate(deck_random)}')

Recomendación random: [195, 195, 315, 315, 405, 405, 555, 564, 564, 614, 614, 662, 662, 825, 825, 1080, 1084, 1783, 1783, 1927, 1927, 2064, 2064, 2078, 2275, 2275, 2571, 2571, 38710, 38710]
Win rate random: 54.78474044799805


In [335]:
deck_most_popular = most_popular_deck() # generamos recomendación más popular
print(f'Recomendación más popular: {deck_most_popular}')
print(f'Win rate más popular: {win_rate(deck_most_popular)}')

Recomendación más popular: [77, 315, 315, 405, 405, 555, 564, 564, 614, 635, 662, 662, 825, 825, 1004, 38547, 38547, 38725, 38725, 38857, 38859, 38859, 38863, 38863, 38868, 38868, 38900, 38900, 39715, 39715]
Win rate más popular: 53.383628845214844


In [ ]:
id_random, deck_random = random_deck()
deck_best_similar = best_similar_deck(deck_random, 25) # generamos recomendación mejor similar
print(f'Recomendación mejor similar: {deck_best_similar}')
print(f'Win rate mejor similar: {win_rate(deck_best_similar)}')

Win rate mejor similar: 56.32284164428711
Recomendación mejor similar: [77, 315, 315, 405, 405, 555, 564, 564, 614, 635, 662, 662, 825, 825, 1004, 38547, 38547, 38725, 38725, 38857, 38859, 38859, 38863, 38863, 38868, 38868, 38900, 38900, 39715, 39715]
Win rate mejor similar: 53.383628845214844


In [ ]:
deck = np.zeros(N, dtype=int)
for i in range(N):
    for card in original_deck:
        if card == cards[i]:
            deck[i] = 1
zeros = [index for index, value in enumerate(deck) if value == 0]
ones = [index for index, value in enumerate(deck) if value == 1]
new_ones = rd.sample(zeros,D-len(ones))
for i in new_ones:
    deck[i] = 1
cards_subset = cards[:N]
deck_our_rec = deck_rec(deck,cards_subset)
print(f'Nuestra recomendación : {binary_to_cards(deck_our_rec, cards_subset)}')
print(f'Win rate nuestra recomendación: {win_rate(binary_to_cards(deck_our_rec, cards_subset))}')

[1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1
 0 1 1]
Nuestra recomendación : [315, 555, 405, 614, 825, 564, 1783, 457, 2275, 1084, 195, 38710, 172, 2874, 621, 1080, 2064, 1801, 1927, 1793, 2572, 2262, 251, 113, 749, 2875, 430, 2037, 2050, 2078]
Win rate nuestra recomendación: 60.91288375854492


Observamos que de los sistemas recomendadores anteriores el que entrega el mazo con un mejor *win rate* es el ...

### Distintos mazos
Ahora, vamos a implementar lo anterior para múltiples mazos aleatorios y volver a comparar.

In [338]:
simulations = {"initial":[], "random":[],"MP":[],"BS":[],"DeckReck":[]}
for i in range(20):
    initial_deck = random_deck()[1]
    simulations["initial"].append(win_rate(initial_deck))
    # random
    simulations["random"].append(win_rate(random_deck()[1]))
    # most popular 
    simulations["MP"].append(win_rate(most_popular_deck()))
    # best similar
    simulations["BS"].append(win_rate(best_similar_deck(initial_deck, 25)))
    deck = np.zeros(N, dtype=int)
    for i in range(N):
        for card in original_deck:
            if card == cards[i]:
                deck[i] = 1
    zeros = [index for index, value in enumerate(deck) if value == 0]
    ones = [index for index, value in enumerate(deck) if value == 1]
    new_ones = rd.sample(zeros,D-len(ones))
    for i in new_ones:
        deck[i] = 1
    cards_subset = cards[:N]
    deck_our_rec = deck_rec(deck,cards_subset)
    simulations["DeckReck"].append(win_rate(binary_to_cards(deck_our_rec, cards_subset)))

In [346]:
print(simulations["initial"])
sim_fl = dict()
for key in simulations.keys():
    sim_fl[key] = [float(simulations[key][i]) for i in range(len(simulations[key]))]

[np.float32(53.06286), np.float32(51.93438), np.float32(57.69546), np.float32(54.63438), np.float32(53.817204), np.float32(54.63331), np.float32(51.93438), np.float32(56.34066), np.float32(53.175045), np.float32(54.633316), np.float32(50.7918), np.float32(56.32461), np.float32(52.291637), np.float32(54.78474), np.float32(49.24304), np.float32(56.342117), np.float32(53.42384), np.float32(56.324768), np.float32(57.38224), np.float32(54.78474)]


In [348]:
#guardando resultados
with open('comparacion_modelos.json', 'w') as json_file:
    json.dump(sim_fl, json_file, indent=4)

In [356]:
comparacion = pd.DataFrame(sim_fl)
comparacion#.describe()

,initial,random,MP,BS,DeckReck
0,53.062859,54.784740,53.383629,53.383629,59.073711
1,51.934380,47.957829,53.383629,53.383629,59.073711
2,57.695461,54.963871,53.383629,56.324768,59.073711
3,54.634380,56.324768,53.383629,56.324768,59.073711
4,53.817204,56.324768,53.383629,53.383629,58.977875
5,54.633308,52.291637,53.383629,53.383629,58.443111
6,51.934380,54.784740,53.383629,53.383629,60.453484
7,56.340660,53.613392,53.383629,56.324768,59.073711
8,53.175045,56.324768,53.383629,51.934380,60.453484
9,54.633316,54.784740,53.383629,56.324768,57.703014


In [357]:
aumentos = pd.DataFrame({"random":comparacion["random"]/comparacion["initial"],
                         "MP": comparacion["MP"]/comparacion["initial"],
                         "BS": comparacion["BS"]/comparacion["initial"],
                         "DeckReck": comparacion["DeckReck"]/comparacion["initial"]})
aumentos

,random,MP,BS,DeckReck
0,1.032450,1.006045,1.006045,1.113278
1,0.923431,1.027905,1.027905,1.137468
2,0.952655,0.925266,0.976243,1.023888
3,1.030940,0.977107,1.030940,1.081255
4,1.046594,0.991944,0.991944,1.095893
5,0.957138,0.977126,0.977126,1.069734
6,1.054884,1.027905,1.027905,1.164036
7,0.951593,0.947515,0.999718,1.048509
8,1.059233,1.003923,0.976668,1.136877
9,1.002772,0.977126,1.030960,1.056187


In [358]:
aumentos.describe()

,random,MP,BS,DeckReck
count,20.000000,20.000000,20.000000,20.000000
mean,1.005203,0.986966,1.001695,1.100204
std,0.053571,0.041468,0.041912,0.045327
min,0.923431,0.925266,0.930316,1.023888
25%,0.955163,0.947784,0.975788,1.072162
50%,1.010616,0.977126,0.999860,1.088574
75%,1.045375,1.009755,1.028318,1.132906
max,1.112538,1.084085,1.112538,1.199636


### Distintos parámetros

In [ ]:
simulations_delta = {"initial":[], "25":[],"20":[],"15":[],"10":[]}
for i in range(20):
    initial_deck = random_deck()[1]
    simulations_delta["initial"].append(win_rate(initial_deck))
    simulations_delta["25"].append(win_rate(best_similar_deck(initial_deck, 25)))
    simulations_delta["20"].append(win_rate(best_similar_deck(initial_deck, 20)))
    simulations_delta["15"].append(win_rate(best_similar_deck(initial_deck, 15)))
    simulations_delta["10"].append(win_rate(best_similar_deck(initial_deck, 10)))
print(simulations["initial"])
sim_delta_fl = dict()
for key in simulations_delta.keys():
    sim_delta_fl[key] = [float(simulations_delta[key][i]) for i in range(len(simulations_delta[key]))]
with open('comparacion_delta.json', 'w') as json_file:
    json.dump(sim_delta_fl, json_file, indent=4)
comparacion_delta = pd.DataFrame(simulations_delta)
comparacion_delta

[np.float32(53.06286), np.float32(51.93438), np.float32(57.69546), np.float32(54.63438), np.float32(53.817204), np.float32(54.63331), np.float32(51.93438), np.float32(56.34066), np.float32(53.175045), np.float32(54.633316), np.float32(50.7918), np.float32(56.32461), np.float32(52.291637), np.float32(54.78474), np.float32(49.24304), np.float32(56.342117), np.float32(53.42384), np.float32(56.324768), np.float32(57.38224), np.float32(54.78474)]


,initial,25,20,15,10
0,48.295925,53.383629,55.183399,48.295925,48.295925
1,54.784740,53.383629,54.784740,54.784740,54.784740
2,52.291637,53.383629,53.175045,55.183399,52.291637
3,54.633308,51.934380,56.324768,54.633308,54.633308
4,56.342117,53.383629,54.963871,56.342117,56.342117
5,52.291637,53.383629,51.934380,51.934380,52.291637
6,54.963871,53.383629,54.784740,56.324768,54.963871
7,50.713581,53.383629,53.383629,54.643101,50.713581
8,53.423840,53.383629,52.274773,54.963871,53.423840
9,56.324768,56.324768,52.274773,56.324612,56.324768


In [367]:
aumentos_delta = pd.DataFrame({"25":comparacion_delta["25"]/comparacion_delta["initial"],
                         "20": comparacion_delta["20"]/comparacion_delta["initial"],
                         "15": comparacion_delta["15"]/comparacion_delta["initial"],
                         "10": comparacion_delta["10"]/comparacion_delta["initial"]})
aumentos_delta.describe()

,25,20,15,10
count,20.000000,20.000000,20.000000,20.0
mean,1.004063,1.003227,1.010305,1.0
std,0.038511,0.050672,0.021856,0.0
min,0.947491,0.883712,0.993168,1.0
25%,0.974425,0.989559,1.000000,1.0
50%,1.001489,1.000000,1.000000,1.0
75%,1.021852,1.020065,1.008421,1.0
max,1.105344,1.142610,1.077485,1.0
